#Data 04: Super store
* Super store 데이터는 국제 배송이 가능한 판매점의 데이터입니다.
*   Data from: https://www.kaggle.com/rohitsahoo/sales-forecasting

##1.데이터 둘러보기

In [ ]:
#한글 폰트 설정하기
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf
#런타임 다시 시작

In [ ]:
#기본 패키지 불러오기
import math
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

plt.style.use('seaborn')
sns.set(font_scale=1)
plt.rc('font', family='NanumBarunGothic') 
plt.rcParams['font.family'] = 'NanumGothic'

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
#구글 드라이브 마운트하기(이미 되어있다면 건너뛴다)
from google.colab import drive 
drive.mount('/content/drive')

In [ ]:
#데이터 불러오기
sp = pd.read_excel('/content/drive/MyDrive/Python/04_Superstore/US Superstore data.xls')

In [ ]:
#불러온 데이터 일단 보기
sp.head(5)

<컬럼 설명>

* Row ID: 일련번호
* Order ID: 주문코드(주문시마다 생성되는 트랜잭션 ID)
* Order Date: 주문일자	
* Ship Date: 배송일자
* Ship Mode: 선적 유형(1st / 2nd / Standard class)
* Customer ID: 고객ID	
* Customer Name: 고객 성명
* Segment: 주문 세그먼트(Consumer / Corpoerate / Home office)
* Country: 주문 국가(모두 US)
* City: 주문 도시
* State: 주문 주
* Postal Code: 주문 우편번호
* Region: 주문 지역(Central / East / South / West)
* Product ID: 상품코드
* Category: 카테고리-대분류
* Sub-Category: 카테고리-중분류
* Product Name: 상품명
* Sales: 판매가
* Quantity: 판매량
* Discount: 할인가
* Profit: 마진


* 범주형 변수: *빈도frequency 계산 가능*
               orderid, ship mode, customer id, customer name, segment,
               country, city, state, postal code, region, product id, category, sub-category, product name
* 연속형 변수: *평균, 표준편차, min/max 등 수치 계산 가능*
               시간변수(orderdate, shipdate), sales, suqntity, discount, profit

In [ ]:
#데이터 기본 정보 확인하기
sp.info()

In [ ]:
#연속형(수치) 데이터의 통계치 확인하기
sp.describe()

In [ ]:
#범주형 데이터의 통계치 확인하기
sp.describe(include=np.object)

In [ ]:
#데이터 내용 정리해서 출력하기
print("총 데이터 수: ", sp.shape[0]*sp.shape[1])
print("총 결측치 수: {} = 전체 데이터의 {:.2f}% ".format(sp.isnull().sum().sum(), (sp.isnull().sum().sum()*100)/(sp.shape[0]*sp.shape[1])))
print("데이터 기간: {} ~ {} ".format(min(sp['Order Date']), max(sp['Order Date'])))
print("전체 판매 물건 수: ", sp['Product ID'].nunique())

##2.질문하기

* 어떤 종류의 물건(sub-category)이 가장 많이 팔렸을까?
* 어느 도시에서 주문량이 가장 많았을까?
* 세그먼트와 지역에 따른 주문량과 판매금액은?
* 할인률이 높을수록 마진은 낮을까?
* 어떤 종류의 상품이 매출이 가장 높을까?
* 지도 위에 판매량을 나타낼 수 있을까?

##3.데이터 정비하기

###3-1. 중복 데이터 처리하기

In [ ]:
#중복이 있나? 없다!
sp.duplicated().sum()

###3-2. 컬럼명 바꾸기

In [ ]:
#컬럼명 확인하기
sp.columns

In [ ]:
#복잡한 컬럼명 단순하게 변경하기
sp.columns = ['row_id','order_id','order_date','ship_date','ship_mode','cust_id','cust_name','seg'
,'country','city','state','post_code','region','product_id','category','subcategory','product_name'
,'sales','quantity','discount','profit']

In [ ]:
sp.columns

###3-3.필요없는 컬럼 제거하기

In [ ]:
#우편번호 컬럼은 사용할 것 같지 않으므로 제거
sp1 = sp.drop(['post_code'],axis=1)
sp1.info()

##4.EDA & Visualization

###4-1. 간단한 분석

In [ ]:
#주문 순위 Top 10인 상품
sp1['product_name'].value_counts().head(10)

In [ ]:
#Violin plot으로 카테고리별 판매금액과 할인율의 분포 확인하기
f, ax = plt.subplots(1, 2, figsize=(13, 7))

#violin plot
#bot plot과 밀도함수가 합쳐진 형태!
sns.violinplot(x=sp['category'], y=sp1['sales'], ax=ax[0])
sns.violinplot(x=sp['category'], y=sp1['discount']*100, ax=ax[1])

In [ ]:
#sub-category 기준으로 어떤 것이 가장 주문량이 많은가? 비율은?
#pie graph로 나타내자!

plt.figure(figsize=(12,10))

sp['subcategory'].value_counts().plot.pie(autopct="%1.1f%%")

plt.show()

###4-2. 주 별 주문량 확인하기

In [ ]:
#주문량이 많은 도시 10개
top_cities= sp['city'].value_counts().nlargest(20)
top_cities

In [ ]:
#주문량이 많은 도시를 그래프로 그려보자
f, ax = plt.subplots(1, 1, figsize=(18, 8))

g = sns.countplot(sp1['state'].sort_values(), ax=ax)
g.set_xticklabels(g.get_xticklabels(),rotation=90)
g.set_title('State', size=15)
g.set_ylabel('')


plt.show()

###4-3. 세그먼트/지역별 주문량

In [ ]:
#세그먼트 별, 지역별 주문량
#Consumer seg가 가장 주문량이 많고, 서부의 주문량이 가장 많다
f, ax = plt.subplots(1, 2, figsize=(14, 6))

sns.countplot(x=sp1['seg'], palette ='magma', ax=ax[0])
ax[0].set_title('Segment')

sns.countplot(x=sp1['region'], palette ='magma', ax=ax[1])
ax[1].set_title('Region')
ax[1].set_ylabel('')

plt.show()

###4-4. 할인률에 따른 마진

In [ ]:
#sns.lineplot은 추정 회귀선과 신뢰구간을 같이 확인할 수 있다
#면적은 신뢰 구간을 나타낸다
#estimator = None으로 두면 x값에 y가 여러개이므로 이상한 그래프가 나온다!
f, ax = plt.subplots(1, 1, figsize=(10, 6))

sns.lineplot('discount', 'profit', data = sp1, color = 'r', ax=ax)

ax.set_xlabel('Discount (%)')
ax.set_ylabel('profit ($)')

plt.show()

###4-5. 상관관계 Heatmap 그리기

In [ ]:
#상관관계 구하기(수치형 변수 사이의 상관관계를 모두 계산해 준다)
sp_corr = sp1[['sales','quantity','discount','profit']].corr()
sp_corr

In [ ]:
#상관관계를 열지도(Heat map)으로 나타내자

f, ax = plt.subplots(1, 1, figsize=(9, 8))
sns.heatmap(sp_corr, annot=True, cmap='YlGnBu', ax=ax)
plt.title('판매가, 판매량, 할인가, 마진의 상관관계', size=15)

#판매가가 높을수록 마진이 높음(당연히)
#할인율이 높을수록 마진은 적음

###4-6. 판매 금액에 따른 상위 10위 품목

In [ ]:
#상품명으로 집계했을 때, 매출액 높은 순 상위 10개
top_prd1 = sp1.groupby(['product_name']).sum().sort_values('sales',ascending=False).head(10)

In [ ]:
#reset index로 다시 새로운 데이터 프레임을 만들어준다
top_prd1.reset_index(inplace=True)
top_prd1

In [ ]:
#Pie 그래프로 상위 10개 품목의 판매금액에 대한 비율 나타내기
f, ax = plt.subplots(1, 1, figsize=(9, 8))

ax.pie(top_prd1['sales'], labels=top_prd1['product_name'], autopct="%1.1f%%", startangle=0)
ax.set_ylabel('')

#흰색 원 덧대기
center_circle = plt.Circle((0,0), 0.5, fc='white')
fig = plt.gcf() #그래프 상의 위치를 위한 함수
fig.gca().add_artist(center_circle)

ax.axis('equal') #파이그래프와 원이 중심축 같도록 지정  
label = ax.annotate('판매금액에 따른\n 상위 10개 품목', color='black', xy=(0, -0.07), fontsize=15, ha="center")

plt.show()

###4-7. 주 별 총 판매금액을 지도 위에 반응형으로 나타내기 

In [ ]:
#주 별로 판매금액(sales) 내용을 지도 위에 나타내보자
#주 별로 판매금액을 집계하자
sales = sp1.groupby(['state']).sum().sort_values('sales', ascending=False)
sales.reset_index(level=0, inplace=True)
#sales = sales.sort_values('state', ascending=True)
sales

In [ ]:
#워싱턴 D.C.는 제거해주기
dc = sales[sales['state'] == 'District of Columbia'].index
sales = sales.drop(dc)
sales

In [ ]:
#그림을 그리기 위해 state_code를 붙여준다
state = ['Alabama', 'Arizona' ,'Arkansas', 'California', 'Colorado', 'Connecticut', 'Delaware', 'Florida', 
         'Georgia', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland',
         'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana','Nebraska', 'Nevada', 'New Hampshire',
         'New Jersey', 'New Mexico', 'New York', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania',
         'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington',
         'West Virginia', 'Wisconsin','Wyoming']
state_code = ['AL','AZ','AR','CA','CO','CT','DE','FL','GA','ID','IL','IN','IA','KS','KY','LA','ME','MD','MA',
              'MI','MN','MS','MO','MT','NE','NV','NH','NJ','NM','NY','NC','ND','OH','OK','OR','PA','RI','SC','SD','TN',
              'TX','UT','VT','VA','WA','WV','WI','WY']

#데이터프레임을 만들어서 stte_cd와 주 이름을 붙여준다
state_cd = pd.DataFrame(state, state_code)
state_cd
state_cd.reset_index(inplace=True)
state_cd.columns = ['state_cd','state']
state_cd

In [ ]:
#state_cd를 붙여주기 위해 sales 데이터프레임을 State 순으로 정렬한 후 인덱스를 새로 매겨준다
sales= sales.sort_values('state', ascending=True)
sales.reset_index(inplace = True)
sales.insert(1, 'state_cd', state_cd['state_cd'])
sales

In [ ]:
#기존 인덱스는 떼버리기
sales.drop('index',1,inplace = True)
sales
#state 옆에 state code가 잘 붙었는지 확인하자

In [ ]:
#plotly를 사용해서 반응형 그래프를 그려보자
import plotly.express as px
  
fig = px.choropleth(locations=sales['state_cd'], locationmode="USA-states", color=sales['sales'], scope="usa"
, color_continuous_scale='peach', title='주 별 총 판매금액')

fig.show()

##5.Review

* Online Retail 데이터는:  
총 데이터 수:  209874
총 결측치 수: 0 = 전체 데이터의 0.00% 
데이터 기간: 2014-01-03 00:00:00 ~ 2017-12-30 00:00:00 
전체 판매 물건 수:  1862


* 어떤 종류의 물건(sub-category)이 가장 많이 팔렸을까?  
  : Binder가 15.2%로 가장 많이 팔렸다
* 어느 도시에서 주문량이 가장 많았을까?  
  : 캘리포니아, 뉴욕, 텍사스 순으로 주문량이 많았다
* 세그먼트와 지역에 따른 주문량은?  
  : Consumer seg가 가장 주문량이 많고, 서부의 주문량이 가장 많다
* 할인률이 높을수록 마진은 낮을까?  
  : 할인률이 50%일 때 마진이 가장 낮다!
* 어떤 종류의 상품이 매출이 가장 높을까?
  : Canon imageCLASS 2200 Advanced Copier의 판매금액이 가장 높다!
* 지도 위에 판매량을 나타낼 수 있을까?